In [57]:
import yaml
import json
import os

# see: https://github.com/fbrandt/ACLPP/blob/master/doc/format.md
# Put the yaml files in the input directory (scenario: base)
INPUT_PATH = './Data/Data_1/0_Input'
OUTPUT_PATH = './Data/Data_1/1_AeULDs/Ulds_scenario_1'

In [58]:
def createOutputFolderIfNotExists():
    isExist = os.path.exists(OUTPUT_PATH)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(OUTPUT_PATH)
       print("The new directory is created!")

In [59]:
def readYaml(filePathImport):
    with open(filePathImport, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [60]:
def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        json_file = json.load(json_file)
        return json_file

In [61]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [62]:
def createItemLabels(baseLabel, labelDic):
    appendix = 0
    if baseLabel in labelDic.keys():
        appendix = labelDic[baseLabel] + 1
    
    labelDic[baseLabel] = appendix
    
    return baseLabel + "-" + str(appendix)

In [63]:
def getWeightFromShipment(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return shipments[shipLabel]['pieces'][piece]['weight']

In [64]:
def getRotationAxes(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    
    # rotation axes is unsed
    switcher = {
        1: "NONE",
        2: "X",
        4: "Y",
        5: "Y",
        8: "Z",
        16: "XZ",
        32: "YZ",
        63: "XYZ",
    }
    
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return switcher.get(shipments[shipLabel]['pieces'][piece]['allowed_rotations'], 'NONE')

In [65]:
def getLoadCapacity(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return shipments[shipLabel]['pieces'][piece]['stack_height'] * sourceItem['lat'] * sourceItem['lng'] 
    

In [66]:
def enrichItem(sourceItem, item, sequence, labelDic, shipments):
    item['itemLabel'] = createItemLabels(sourceItem['piece'], labelDic)
    shape = {
        'shapeType' : 'Box',
        # Lat according to Brandt is the width in our system
        'width': sourceItem['lat'],
        'height': sourceItem['height'],
        'depth': sourceItem['lng'],
        'rotationState': 0,
        'id': 2025, # unused
        'volume': sourceItem['lat'] * sourceItem['height'] * sourceItem['lng']
    }
    item['shape'] = shape
    item['x'] = sourceItem['start_lat']
    item['z'] = sourceItem['start_lng']
    item['y'] = sourceItem['start_height']
    item['shipmentLabel'] = sourceItem['shipment']
    
    if item['shipmentLabel'] == "":
        print(item['itemLabel'])
        raise(False)
    
    item['sequence'] = sequence
    item['loadCapacity'] = getLoadCapacity(sourceItem, shipments)
    item['rotationAxes'] = getRotationAxes(sourceItem, shipments)
    item['weight'] = getWeightFromShipment(sourceItem, shipments)
    
    return item

In [67]:
def readUldProperties(sourceUldType):
    mapping = readJson("./01_masterdata/0_ulds/mapping_uld_properties.json")
    if sourceUldType in mapping.keys():
        uldTypeName = mapping[sourceUldType]
        return readJson("./01_masterdata/0_ulds/" + uldTypeName + ".json") 
    else:
        
        sourceUldTypeTrim = sourceUldType[:sourceUldType.rfind("_")]
        
        if sourceUldTypeTrim in mapping.keys():
            uldTypeName = mapping[sourceUldTypeTrim]
            return readJson("./01_masterdata/0_ulds/" + uldTypeName + ".json")
        
        else:
            print(sourceUldType)


In [68]:
def createULD(sourceUld, labelDic, shipments):
    newUld = {"placedItems":[], "id":1}
    newUld["properties"] = readUldProperties(sourceUld["uld_type"])["properties"]
    i = 0
    for sourceItem in sourceUld['loaded']:
        item = readJson('./01_masterdata/1_items/placeholders_placed_item.json')
        item = enrichItem(sourceItem, item, i, labelDic, shipments)
        i = i + 1
        newUld['placedItems'].append(item)
    return newUld

In [69]:
## run through flights
def transformFile(sourceFile):
    sourceFile = readYaml(sourceFile)
    for flightKey, segment in sourceFile['segments'].items():
        shipments = segment['shipments']
        for uldKey, sourceUld in segment['built_ulds'].items():
            path_output_full = OUTPUT_PATH + '/' + flightKey + "_" + uldKey + ".json"
            labelDic = {}
            newUld = createULD(sourceUld, labelDic, shipments)
            writeJson(path_output_full, newUld)

In [71]:
from tqdm import tqdm


def execute():
    createOutputFolderIfNotExists()

    for f in tqdm(os.listdir(INPUT_PATH)):
        transformFile(INPUT_PATH + "/" + f)

In [72]:
execute()